In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import pickle
import pandas as pd
import numpy as np
from columns_list2 import *

In [2]:
with open("rcms_business_new.pkl","rb") as f:
    df=pickle.load(f)

In [3]:
df.head(5)

,RCMS_BSNS_ID,PMS_ID,BSNS_CL_CD,BSNS_NM,BUCL_DSCR,HIRK_BSNS_CL_CD,BUCL_SUMM_NM,BUCL_CD_USE_YN,FRST_REG_USER_ID_x,FRST_REG_DT_x,...,BD_AR_VL,RGT_INFR_YN,CLTR_OFR_YN,BSC_PTIM,MODF_DE,NEW_BSTP_CD,FRST_REG_USER_ID_y,FRST_REG_DT_y,LAST_MODF_USER_ID_y,LAST_MODF_DT_y
0,B20171520071,G01,B00325,국제상호인정시험평가능력기반구축,국제상호인정시험평가능력기반구축,B10081,국제상호인정시험평가능력기반구축,Y,Migration(2017),17/11/02,...,"14,471㎡(4,377평)",1.0,N,20/01/10,2020-01-11,M70129,Migration(2017),2015-08-31 20:20:46,PGM-BAT-026-01,2020-01-14 06:40:09
1,B20171520071,G01,B00325,국제상호인정시험평가능력기반구축,국제상호인정시험평가능력기반구축,B10081,국제상호인정시험평가능력기반구축,Y,Migration(2017),17/11/02,...,"14,471㎡(4,377평)",1.0,N,20/01/10,2020-01-11,M70129,Migration(2017),2015-08-31 20:20:46,PGM-BAT-026-01,2020-01-14 06:40:09
2,B20171520071,G01,B00325,국제상호인정시험평가능력기반구축,국제상호인정시험평가능력기반구축,B10081,국제상호인정시험평가능력기반구축,Y,Migration(2017),17/11/02,...,"14,471㎡(4,377평)",1.0,N,20/01/10,2020-01-11,M70129,Migration(2017),2015-08-31 20:20:46,PGM-BAT-026-01,2020-01-14 06:40:09
3,B20171520071,G01,B00325,국제상호인정시험평가능력기반구축,국제상호인정시험평가능력기반구축,B10081,국제상호인정시험평가능력기반구축,Y,Migration(2017),17/11/02,...,"14,471㎡(4,377평)",1.0,N,20/01/10,2020-01-11,M70129,Migration(2017),2015-08-31 20:20:46,PGM-BAT-026-01,2020-01-14 06:40:09
4,B20171520071,G01,B00325,국제상호인정시험평가능력기반구축,국제상호인정시험평가능력기반구축,B10081,국제상호인정시험평가능력기반구축,Y,Migration(2017),17/11/02,...,"14,471㎡(4,377평)",1.0,N,20/01/10,2020-01-11,M70129,Migration(2017),2015-08-31 20:20:46,PGM-BAT-026-01,2020-01-14 06:40:09


In [64]:
df_conti=df[DATA_CLASS_DICT["continous"]]
df_code=df[DATA_CLASS_DICT["code"]]
df["F"]=df["DSAPVL_HDLG_STT_CD"].apply(lambda x:0 if x=="B15010" else 1)

## random forest

In [342]:
rfc=RandomForestClassifier(class_weight={0:1,1:20},n_estimators=500)

In [343]:
rfc.fit(df_conti,df["F"])

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 20},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [344]:
pred=rfc.predict(df_conti)

In [345]:
from sklearn.metrics import confusion_matrix

In [346]:
confusion_matrix(df["F"],pred)

array([[4085,   56],
       [   1,   15]], dtype=int64)

In [318]:
fi_list=[]
for col,imp_score in zip(df_conti.columns,rfc.feature_importances_):
    fi_list.append([col,imp_score])

In [319]:
fi_list.sort(key=lambda x:x[1])

In [320]:
fi_list.reverse()

In [321]:
fi_list

[['USE_SPLY_AMT', 0.28102652221441166],
 ['EXCT_AMT', 0.23620171532834022],
 ['USE_SPLY_AMT_1', 0.23436607788301156],
 ['USE_VAT_AMT_1', 0.03993422783324027],
 ['USE_VAT_AMT', 0.03906303064635284],
 ['인건비내역수', 0.02571993921123462],
 ['ORGN_TTL_BSNS_AMT', 0.02256921420676373],
 ['GOV_CTRB_AMT_1', 0.021803639855787524],
 ['AGRT_BNDS_AMT', 0.021752879805179055],
 ['PRVT_ALOT_SPOT_AMT_1', 0.020364774648309384],
 ['ATDT_CNT', 0.018728808102764494],
 ['STMT_SPLY_AMT', 0.008149401696150442],
 ['STAC_MM', 0.0073213687047014395],
 ['STMT_VAT_AMT', 0.004287842637850797],
 ['PRVT_ALOT_SPOT_AMT', 0.003577534280179542],
 ['VAT_HDLG_RSTO_AMT', 0.0032084833535643157],
 ['ANNL', 0.003031034638210722],
 ['PRVT_ALOT_CASH_AMT', 0.0029094837874166996],
 ['GOV_CTRB_AMT', 0.002574764340935954],
 ['TTL_BSNS_AMT', 0.002198595296502423],
 ['RSTO_CONF_SPLY_AMT_1', 0.0004027547445238426],
 ['PRYY_CASH_CROV_AMT', 0.0003665838116424006],
 ['RVN_AMT', 0.00025291560461192014],
 ['STNG_AMT', 9.286104475554948e-05],
 

## gradient boosting

In [159]:
gbc=GradientBoostingClassifier(n_estimators=500,learning_rate=0.5)

In [160]:
gbc.fit(df_conti,df["F"])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.5, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [161]:
pred=gbc.predict(df_conti)

In [162]:
confusion_matrix(df["F"],pred)

array([[4141,    0],
       [  10,    6]], dtype=int64)

In [165]:
fi_list2=[]
for col,imp_score in zip(df_conti.columns,gbc.feature_importances_):
    fi_list2.append([col,imp_score])

In [166]:
fi_list2.sort(key=lambda x:x[1])
fi_list2.reverse()
fi_list2

[['USE_SPLY_AMT', 0.3858884547096243],
 ['EXCT_AMT', 0.29203024841611186],
 ['USE_SPLY_AMT_1', 0.16790458653256368],
 ['USE_VAT_AMT', 0.0458903570972709],
 ['ORGN_TTL_BSNS_AMT', 0.041756536099537396],
 ['USE_VAT_AMT_1', 0.03032159381614684],
 ['AGRT_BNDS_AMT', 0.005558041043258521],
 ['ATDT_CNT', 0.0034620488138102177],
 ['PRVT_ALOT_CASH_AMT', 0.0033323816394103234],
 ['GOV_CTRB_AMT', 0.0028586035505199546],
 ['VAT_HDLG_RSTO_AMT', 0.0027898943836859665],
 ['PRYY_CASH_CROV_AMT', 0.002705452348900595],
 ['GOV_CTRB_AMT_1', 0.0024976638359671933],
 ['TTL_BSNS_AMT', 0.0023932528987340856],
 ['STMT_VAT_AMT', 0.0023795590116341262],
 ['PRVT_ALOT_SPOT_AMT', 0.0021273327198306673],
 ['PRVT_ALOT_SPOT_AMT_1', 0.002087685747500549],
 ['STMT_SPLY_AMT', 0.0019969654296615024],
 ['ANNL', 0.0018626994917280138],
 ['인건비내역수', 0.00015663542558032456],
 ['STAC_MM', 6.988522945659553e-09],
 ['STNG_AMT', 0.0],
 ['RSTO_CONF_SPLY_AMT_1', 0.0],
 ['RSTO_CONF_SPLY_AMT', 0.0],
 ['USE_CROV_VAT_AMT', 0.0],
 ['USE_C

## AdaBoost 

In [83]:
abc=AdaBoostClassifier(n_estimators=500,learning_rate=1)

In [84]:
abc.fit(df_conti,df["F"])

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=500, random_state=None)

In [85]:
pred=abc.predict(df_conti)

In [86]:
confusion_matrix(df["F"],pred)

array([[4141,    0],
       [  10,    6]], dtype=int64)

In [87]:
fi_list3=[]
for col,imp_score in zip(df_conti.columns,abc.feature_importances_):
    fi_list3.append([col,imp_score])

In [88]:
fi_list3.sort(key=lambda x:x[1])
fi_list3.reverse()
fi_list3

[['USE_SPLY_AMT', 0.528],
 ['EXCT_AMT', 0.216],
 ['USE_SPLY_AMT_1', 0.11],
 ['PRVT_ALOT_SPOT_AMT_1', 0.032],
 ['USE_VAT_AMT', 0.022],
 ['AGRT_BNDS_AMT', 0.022],
 ['USE_VAT_AMT_1', 0.018],
 ['GOV_CTRB_AMT_1', 0.018],
 ['ATDT_CNT', 0.008],
 ['ORGN_TTL_BSNS_AMT', 0.008],
 ['PRVT_ALOT_CASH_AMT', 0.008],
 ['인건비내역수', 0.002],
 ['STMT_SPLY_AMT', 0.002],
 ['RSTO_CONF_SPLY_AMT_1', 0.002],
 ['PRVT_ALOT_SPOT_AMT', 0.002],
 ['TTL_BSNS_AMT', 0.002],
 ['STAC_MM', 0.0],
 ['STNG_AMT', 0.0],
 ['STMT_VAT_AMT', 0.0],
 ['RSTO_CONF_SPLY_AMT', 0.0],
 ['VAT_HDLG_RSTO_AMT', 0.0],
 ['USE_CROV_VAT_AMT', 0.0],
 ['USE_CROV_SPLY_AMT', 0.0],
 ['PRYY_CASH_CROV_AMT', 0.0],
 ['RVN_AMT', 0.0],
 ['GOV_CTRB_AMT', 0.0],
 ['ANNL', 0.0]]

## XGBoost

In [349]:
xgc=XGBClassifier(learning_rate=1.2,booster="gbtree")

In [350]:
xgc.fit(df_conti,df["F"])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=1.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [351]:
pred=xgc.predict(df_conti)

In [352]:
confusion_matrix(df["F"],pred)

array([[4141,    0],
       [  11,    5]], dtype=int64)

In [353]:
fi_list4=[]
for col,imp_score in zip(df_conti.columns,xgc.feature_importances_):
    fi_list4.append([col,imp_score])

In [354]:
fi_list4.sort(key=lambda x:x[1])
fi_list4.reverse()
fi_list4

[['PRVT_ALOT_CASH_AMT', 0.26632753],
 ['인건비내역수', 0.25874308],
 ['STMT_SPLY_AMT', 0.16000737],
 ['ORGN_TTL_BSNS_AMT', 0.04293435],
 ['ATDT_CNT', 0.042773075],
 ['AGRT_BNDS_AMT', 0.04178966],
 ['USE_SPLY_AMT_1', 0.037646454],
 ['PRVT_ALOT_SPOT_AMT', 0.035293747],
 ['USE_VAT_AMT', 0.029282963],
 ['PRVT_ALOT_SPOT_AMT_1', 0.02698342],
 ['USE_SPLY_AMT', 0.02678092],
 ['EXCT_AMT', 0.021230193],
 ['STAC_MM', 0.0067894254],
 ['USE_VAT_AMT_1', 0.0034178337],
 ['STNG_AMT', 0.0],
 ['STMT_VAT_AMT', 0.0],
 ['RSTO_CONF_SPLY_AMT_1', 0.0],
 ['RSTO_CONF_SPLY_AMT', 0.0],
 ['VAT_HDLG_RSTO_AMT', 0.0],
 ['USE_CROV_VAT_AMT', 0.0],
 ['USE_CROV_SPLY_AMT', 0.0],
 ['PRYY_CASH_CROV_AMT', 0.0],
 ['RVN_AMT', 0.0],
 ['GOV_CTRB_AMT_1', 0.0],
 ['GOV_CTRB_AMT', 0.0],
 ['TTL_BSNS_AMT', 0.0],
 ['ANNL', 0.0]]

## 명목형 데이터

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix

In [13]:
df_code=df[DATA_CLASS_DICT["code"]]

In [93]:
df_code

,BSNS_CL_CD,HIRK_BSNS_CL_CD,SBJT_STEP_CD,BSNS_SPCH_SE_CD,SBJT_PRG_STT_CD,BFCN_BSNS_CL_CD,AGRT_SE_CD,EXE_PRID_SE_CD,AGRT_ORGN_ROLE_SE_CD,NOPRFT_ORGN_YN,...,EXCT_STEP_SE_CD,EXCT_KIND_SE_CD,EXCT_ERR_CD,ORGN_CNLK_DTRS_YN_2,USER_EXCT_STEP_SE_CD,정밀점검대상여부,KED_CD,BSTP_CD,ORGN_CNLK_DTRS_YN_1,STP_FORM_CD_VL
0,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934.0,M70129,N,일반법인
1,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03001,30000,N,2,N,991934.0,M70129,N,일반법인
2,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934.0,M70129,N,일반법인
3,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934.0,M70129,N,일반법인
4,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934.0,M70129,N,일반법인
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1093103.0,K65131,N,일반법인
4153,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1093103.0,K65131,N,일반법인
4154,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1093103.0,K65131,N,일반법인
4155,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1093103.0,K65131,N,일반법인


In [94]:
imputer=SimpleImputer(strategy="most_frequent")
df_code=imputer.fit_transform(df_code)
df_code=pd.DataFrame(df_code,columns=DATA_CLASS_DICT["code"])
df_code

,BSNS_CL_CD,HIRK_BSNS_CL_CD,SBJT_STEP_CD,BSNS_SPCH_SE_CD,SBJT_PRG_STT_CD,BFCN_BSNS_CL_CD,AGRT_SE_CD,EXE_PRID_SE_CD,AGRT_ORGN_ROLE_SE_CD,NOPRFT_ORGN_YN,...,EXCT_STEP_SE_CD,EXCT_KIND_SE_CD,EXCT_ERR_CD,ORGN_CNLK_DTRS_YN_2,USER_EXCT_STEP_SE_CD,정밀점검대상여부,KED_CD,BSTP_CD,ORGN_CNLK_DTRS_YN_1,STP_FORM_CD_VL
0,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934,M70129,N,일반법인
1,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03001,30000,N,2,N,991934,M70129,N,일반법인
2,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934,M70129,N,일반법인
3,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934,M70129,N,일반법인
4,B00325,B10081,0,20,7,S3029,2,CA5020,C,Y,...,1,B03000,30000,N,1,N,991934,M70129,N,일반법인
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1.0931e+06,K65131,N,일반법인
4153,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1.0931e+06,K65131,N,일반법인
4154,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1.0931e+06,K65131,N,일반법인
4155,B00328,B10080,0,10,9,S3060,1,CA5030,E,Y,...,2,B03001,30000,N,2,N,1.0931e+06,K65131,N,일반법인


In [96]:
for col in df_code.columns:
    uniq_val=df_code[col].unique().tolist()
    df_code[col]=df_code[col].apply(lambda x:uniq_val.index(x))    

In [97]:
df_code.head(5)

,BSNS_CL_CD,HIRK_BSNS_CL_CD,SBJT_STEP_CD,BSNS_SPCH_SE_CD,SBJT_PRG_STT_CD,BFCN_BSNS_CL_CD,AGRT_SE_CD,EXE_PRID_SE_CD,AGRT_ORGN_ROLE_SE_CD,NOPRFT_ORGN_YN,...,EXCT_STEP_SE_CD,EXCT_KIND_SE_CD,EXCT_ERR_CD,ORGN_CNLK_DTRS_YN_2,USER_EXCT_STEP_SE_CD,정밀점검대상여부,KED_CD,BSTP_CD,ORGN_CNLK_DTRS_YN_1,STP_FORM_CD_VL
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## random forest

In [47]:
rfc=RandomForestClassifier(class_weight={0:1,1:20},n_estimators=500)
rfc.fit(df_code,df["F"])
pred=rfc.predict(df_code)
confusion_matrix(df["F"],pred)

array([[4120,   21],
       [   8,    8]], dtype=int64)

In [39]:
fi_list=[]
for col,imp_score in zip(df_code.columns,rfc.feature_importances_):
    fi_list.append([col,imp_score])
fi_list.sort(key=lambda x:x[1])
fi_list.reverse()
fi_list

[['DPTC_ITEPD_CD', 0.23272115550963662],
 ['SBJT_AGRT_ORGN_TRSC_YN', 0.15769335857922134],
 ['PTC_ITEPD_CD', 0.11984447453595912],
 ['KED_CD', 0.0767656034097572],
 ['TRSC_EVDC_SE_CD', 0.07352518674720145],
 ['BSTP_CD', 0.03606454073540737],
 ['ORGN_CL_CD', 0.029752663224207616],
 ['EXCT_ERR_CD', 0.02703796364995083],
 ['AGRT_ORGN_ENPR_SCL_CD', 0.024916435540584002],
 ['ORGN_CNLK_DTRS_YN_2', 0.02306555990308122],
 ['EXCT_KIND_SE_CD', 0.018510666647879887],
 ['EXCT_TP_SE_CD', 0.017677853315991577],
 ['USER_EXCT_STEP_SE_CD', 0.016295319499148503],
 ['CARD_STMT_WAY_SE_CD', 0.01519389774313936],
 ['EXCT_STEP_SE_CD', 0.01503328198845343],
 ['AGRT_ORGN_ROLE_SE_CD', 0.014359489901973183],
 ['EXCT_PRG_YN', 0.013659027339128606],
 ['UNIC_AORGN_YN', 0.013388532970308346],
 ['LCOST_ACCT_USE_YN', 0.010149126316011274],
 ['STP_FORM_CD_VL', 0.009946206737429717],
 ['BFCN_BSNS_CL_CD', 0.0090971814471736],
 ['HIRK_BSNS_CL_CD', 0.008190560831471079],
 ['BSNS_CL_CD', 0.007370477825993017],
 ['ORGN_CNLK_

## gradient boosting

In [44]:
gbc=GradientBoostingClassifier(n_estimators=500,learning_rate=1)
gbc.fit(df_code,df["F"])
pred=gbc.predict(df_code)
confusion_matrix(df["F"],pred)

array([[4119,   22],
       [  11,    5]], dtype=int64)

In [46]:
fi_list2=[]
for col,imp_score in zip(df_code.columns,gbc.feature_importances_):
    fi_list2.append([col,imp_score])
fi_list2.sort(key=lambda x:x[1])
fi_list2.reverse()
fi_list2

[['SBJT_AGRT_ORGN_TRSC_YN', 0.46699308498202796],
 ['TRSC_EVDC_SE_CD', 0.16617835056514282],
 ['CRMT_ALLW_YN', 0.12629281464353156],
 ['LCOST_ACCT_USE_YN', 0.12423949943087838],
 ['DPTC_ITEPD_CD', 0.05534128080909563],
 ['PTC_ITEPD_CD', 0.04884518715707322],
 ['KED_CD', 0.007047390399507053],
 ['BFCN_BSNS_CL_CD', 0.0010267725610100806],
 ['AGRT_ORGN_ENPR_SCL_CD', 0.0010267724953482047],
 ['EXCT_ERR_CD', 0.0009256112467661459],
 ['HIRK_BSNS_CL_CD', 0.0005133862814118748],
 ['ORGN_CL_CD', 0.0005133862808330641],
 ['BSNS_CL_CD', 0.000513383561955839],
 ['UNIC_AORGN_YN', 0.0005130286450118547],
 ['USER_EXCT_STEP_SE_CD', 2.7697727850373223e-05],
 ['CARD_STMT_WAY_SE_CD', 2.247086092340334e-06],
 ['ORGN_CNLK_DTRS_YN_2', 1.0149597289007615e-07],
 ['BSTP_CD', 4.630490645457874e-09],
 ['STP_FORM_CD_VL', 0.0],
 ['ORGN_CNLK_DTRS_YN_1', 0.0],
 ['정밀점검대상여부', 0.0],
 ['EXCT_KIND_SE_CD', 0.0],
 ['EXCT_STEP_SE_CD', 0.0],
 ['EXCT_TP_SE_CD', 0.0],
 ['EXCT_PRG_YN', 0.0],
 ['VAT_HDLG_SE_CD', 0.0],
 ['EQMT_RE

## AdaBoost 

In [110]:
abc=AdaBoostClassifier(n_estimators=500,learning_rate=1,random_state=42)
abc.fit(df_code,df["F"])
pred=abc.predict(df_code)
confusion_matrix(df["F"],pred)

array([[4137,    4],
       [  15,    1]], dtype=int64)

In [111]:
fi_list3=[]
for col,imp_score in zip(df_code.columns,abc.feature_importances_):
    fi_list3.append([col,imp_score])
fi_list3.sort(key=lambda x:x[1])
fi_list3.reverse()
fi_list3

[['DPTC_ITEPD_CD', 0.632],
 ['SBJT_AGRT_ORGN_TRSC_YN', 0.102],
 ['KED_CD', 0.058],
 ['ORGN_CNLK_DTRS_YN', 0.052],
 ['BSTP_CD', 0.042],
 ['USER_EXCT_STEP_SE_CD', 0.036],
 ['EXCT_ERR_CD', 0.034],
 ['TRSC_EVDC_SE_CD', 0.014],
 ['BFCN_BSNS_CL_CD', 0.006],
 ['PTC_ITEPD_CD', 0.004],
 ['AGRT_ORGN_ENPR_SCL_CD', 0.004],
 ['BSNS_CL_CD', 0.004],
 ['ORGN_CNLK_DTRS_YN_2', 0.002],
 ['EXCT_KIND_SE_CD', 0.002],
 ['EXCT_TP_SE_CD', 0.002],
 ['VAT_HDLG_SE_CD', 0.002],
 ['UNIC_AORGN_YN', 0.002],
 ['HIRK_BSNS_CL_CD', 0.002],
 ['STP_FORM_CD_VL', 0.0],
 ['ORGN_CNLK_DTRS_YN_1', 0.0],
 ['정밀점검대상여부', 0.0],
 ['EXCT_STEP_SE_CD', 0.0],
 ['EXCT_PRG_YN', 0.0],
 ['EQMT_REG_SYS_SE_CD', 0.0],
 ['USE_AMT_SE_CD', 0.0],
 ['ORGN_CL_CD', 0.0],
 ['LCOST_ACCT_USE_YN', 0.0],
 ['CARD_STMT_WAY_SE_CD', 0.0],
 ['CRMT_ALLW_YN', 0.0],
 ['AGRT_ORGN_CHNG_SE_CD', 0.0],
 ['NOPRFT_ORGN_YN', 0.0],
 ['AGRT_ORGN_ROLE_SE_CD', 0.0],
 ['EXE_PRID_SE_CD', 0.0],
 ['AGRT_SE_CD', 0.0],
 ['SBJT_PRG_STT_CD', 0.0],
 ['BSNS_SPCH_SE_CD', 0.0],
 ['SBJT_ST

## XGBoost

In [112]:
xgc=XGBClassifier(learning_rate=1.2,booster="gbtree")
xgc.fit(df_code,df["F"])
pred=xgc.predict(df_code)
confusion_matrix(df["F"],pred)

array([[4140,    1],
       [  12,    4]], dtype=int64)

In [113]:
fi_list4=[]
for col,imp_score in zip(df_code.columns,xgc.feature_importances_):
    fi_list4.append([col,imp_score])
fi_list4.sort(key=lambda x:x[1])
fi_list4.reverse()
fi_list4

[['PTC_ITEPD_CD', 0.2523521],
 ['EXCT_STEP_SE_CD', 0.1997483],
 ['EXCT_TP_SE_CD', 0.12743388],
 ['CARD_STMT_WAY_SE_CD', 0.10794379],
 ['ORGN_CNLK_DTRS_YN_2', 0.0674451],
 ['SBJT_AGRT_ORGN_TRSC_YN', 0.05435421],
 ['AGRT_ORGN_ENPR_SCL_CD', 0.04037398],
 ['DPTC_ITEPD_CD', 0.023959495],
 ['BSNS_CL_CD', 0.022010393],
 ['AGRT_ORGN_ROLE_SE_CD', 0.02140572],
 ['ORGN_CL_CD', 0.019589605],
 ['KED_CD', 0.018348325],
 ['BSTP_CD', 0.01676124],
 ['LCOST_ACCT_USE_YN', 0.006946078],
 ['USER_EXCT_STEP_SE_CD', 0.0066267876],
 ['EXCT_ERR_CD', 0.005940069],
 ['TRSC_EVDC_SE_CD', 0.0032107048],
 ['EXCT_PRG_YN', 0.0025012891],
 ['UNIC_AORGN_YN', 0.0017622533],
 ['ORGN_CNLK_DTRS_YN', 0.0012867475],
 ['STP_FORM_CD_VL', 0.0],
 ['ORGN_CNLK_DTRS_YN_1', 0.0],
 ['정밀점검대상여부', 0.0],
 ['EXCT_KIND_SE_CD', 0.0],
 ['VAT_HDLG_SE_CD', 0.0],
 ['EQMT_REG_SYS_SE_CD', 0.0],
 ['USE_AMT_SE_CD', 0.0],
 ['CRMT_ALLW_YN', 0.0],
 ['AGRT_ORGN_CHNG_SE_CD', 0.0],
 ['NOPRFT_ORGN_YN', 0.0],
 ['EXE_PRID_SE_CD', 0.0],
 ['AGRT_SE_CD', 0.0],
 

ValueError: 'KED_CD' is not in list